<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2022/ex12notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2022 ex12notebookC

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo12.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2022

----
## 演習問題 - 猫顔画像のクラスタリング
----

猫顔画像131枚をクラスタリングしてみましょう．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# SciPy の階層型クラスタリングパッケージ
import scipy.cluster.hierarchy as hierarchy

# scikit-learn の K-平均法のクラス
from sklearn.cluster import KMeans

---
### 準備

画像を並べて表示する関数を定義しておきます．

In [ ]:
#####  データの最初の nx x ny 枚を可視化
#
def mosaicImage(ax, dat, nx, ny, nrow=64, ncol=64, gap=4):

    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width), dtype=int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        for ix in range(nx):
            if iy*nx+ix >= dat.shape[0]:
                break
            ltx = ix*(ncol + gap) + gap
            img[lty:lty+nrow, ltx:ltx+ncol] = dat[iy*nx+ix, :].reshape((nrow, ncol))

    # 表示
    ax.axis('off')
    ax.imshow(img, cmap='gray')


猫顔画像データを入手して一部を表示してみます．

In [ ]:
# 猫顔画像データを入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/cat131.npz
cat = np.load('cat131.npz')['cat131']
N, D = cat.shape
print(f'データ数 x 次元数 = {N} x {D}')

# 最初の 20 枚を表示
fig, ax = plt.subplots(figsize=(6, 6))
mosaicImage(ax, cat, 5, 4)
plt.show()  

---
### 階層型クラスタリングしてみよう



上記の配列 `cat` に格納されたデータを対象に階層型クラスタリングを実行します．
[scipy.cluster.hiearchy.linkage](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html) でクラスタリングして，
[scipy.cluster.hierarchy.dendrogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html) でデンドログラムを描きます．
標本間の距離はユークリッド距離，クラスタ間の距離はウォード法で測ります．

In [ ]:
link = hierarchy.linkage(cat, method='ward', metric='euclidean')
fig, ax = plt.subplots(figsize=(8, 20))
hierarchy.dendrogram(link, orientation='right', distance_sort='descending', show_leaf_counts=True, leaf_font_size=8, ax=ax)
plt.show()

次のセルを実行すると，`threshold` で指定した値をクラスタ間距離のしきい値として画像をクラスタに分けて，それぞれのクラスタに属する画像の数を表示します．

In [ ]:
threshold = 20000 # クラスタ間距離のしきい値

label = hierarchy.fcluster(link, t=threshold, criterion='distance')
print(label)
K = max(label) # クラスタ数

print()
print(f'threshold = {threshold}, K = {K}')
for ik in range(K):
    print(f'cluster{ik+1}: num = {np.sum(label == ik+1)}')

次のセルを実行すると，上で得られたクラスタリングの結果を画像として表示します．一つのクラスタに属する画像を横に並べてあります．

In [ ]:
# 各クラスタに属する画像を表示

nc = 10 # 1行に並べる画像の最大数

dat = np.zeros((K, nc, D))
for ik in range(K):
    idx = label == (ik+1)
    n = np.sum(idx)
    if n < nc:
        dat[ik, :n, :] = cat[idx, :]
    else:
        dat[ik, :, :] = cat[idx, :][:nc, :]

dat = dat.reshape((-1, D))

fig, ax = plt.subplots(figsize=(12, 12))
mosaicImage(ax, dat, nc, K)
plt.show()  

### 問題1

次のことを考えて／調べてメモしておこう：

(1) この実験では何をやっている？どんな手法を使っている？

(2) 使っているデータのデータ数と次元数はいくつ？


### 問題2

次のことをやろう：

(1) クラスタ間距離のしきい値をいろいろ変えて結果を観察しよう．

(2) クラスタ間距離のしきい値が 6000 のときのクラスタ数と，各クラスタに属する画像の数の最大と最小をメモしよう．

(3) クラスタ数が 13 になるようなクラスタ間距離のしきい値を求め，そのときに各クラスタに属する画像の数の最大と最小をメモしよう．

----
## 演習問題 - 手書き数字画像のクラスタリング
----

手書き数字の画像をクラスタリングしてみましょう．

---
### 準備


In [ ]:
# 手書き数字データの入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/minimnist.npz
rv = np.load('minimnist.npz')
X_train = rv['datL'].astype(float)
#Y_train = rv['labL']
#X_test = rv['datT'].astype(float)
#Y_test = rv['labT']
#print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

#K = 10 # クラス数
N_train, D = X_train.shape
#N_test, _  = X_test.shape

In [ ]:
print('### 手書き数字画像 ###')
print(f'データ数 x 次元数 = {X_train.shape[0]} x {X_train.shape[1]}')

# 最初の 60 枚を表示
fig, ax = plt.subplots(figsize=(12, 6))
mosaicImage(ax, X_train, 15, 4, nrow=28, ncol=28)
plt.show()  

---
### 階層型クラスタリングしてみよう

上記の配列 `X_train` に格納されたデータを対象に階層型クラスタリングを実行します．
[scipy.cluster.hiearchy.linkage](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html) でクラスタリングして，
[scipy.cluster.hierarchy.dendrogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html) でデンドログラムを描きます．
標本間の距離はユークリッド距離，クラスタ間の距離はウォード法で測ります．

In [ ]:
link = hierarchy.linkage(X_train, method='ward', metric='euclidean')
fig, ax = plt.subplots(figsize=(12, 12))
hierarchy.dendrogram(link, ax=ax, truncate_mode='lastp', p=50, distance_sort='descending')
plt.show()

この実験ではデータ数が非常に多く，完全なデンドログラムを描くのは大変です．そのため，上記の図では，クラスター数が50以上になる枝分かれは省略して描いてあります．実際のデンドログラムは，一つのクラスタに一つのデータしか入らないところまで枝分かれしていることに注意．

次の二つのセルを実行すると，猫顔画像の実験と同様にクラスタリング結果を出力させることができます．

In [ ]:
threshold = 10000 # クラスタ間距離のしきい値

label = hierarchy.fcluster(link, t=threshold, criterion='distance')
print(label)
print()
K = max(label) # クラスタ数
print(f'threshold = {threshold}, K = {K}')
for ik in range(K):
    print(f'cluster{ik+1}: num = {np.sum(label == ik+1)}')

In [ ]:
# 各クラスタに属する画像を表示
#    1行が1クラスタ．1行あたり最大 nc 枚を表示

nc = 16

dat = np.zeros((K, nc, D))
for ik in range(K):
    idx = label == (ik+1)
    n = np.sum(idx)
    if n < nc:
        dat[ik, :n, :] = X_train[idx, :]
    else:
        dat[ik, :, :] = X_train[idx, :][:nc, :]

dat = dat.reshape((-1, D))

fig, ax = plt.subplots(figsize=(12, 16))
mosaicImage(ax, dat, nc, K, nrow=28, ncol=28)
plt.show()  

### 問題3

次のことを考えて／調べてメモしておこう：

(1) この実験では何をやっている？どんな手法を使っている？

(2) 使っているデータのデータ数と次元数はいくつ？


### 問題4

次のことをやろう：

(1) クラスタ間距離のしきい値をいろいろ変えて結果を観察しよう．

(2) クラスタ間距離のしきい値が 18800 のときのクラスタ数をメモしよう．